## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-26-17-01-08 +0000,nypost,NYC students struggle to sign in for virtual c...,https://nypost.com/2026/01/26/us-news/nyc-stud...
1,2026-01-26-16-53-40 +0000,nyt,"Private Jet Crashes at Maine Airport, Killing 7",https://www.nytimes.com/2026/01/25/us/bangor-m...
2,2026-01-26-16-52-34 +0000,nyt,"In Xi’s China, Top General’s Fall Shows Precar...",https://www.nytimes.com/2026/01/26/world/asia/...
3,2026-01-26-16-51-52 +0000,bbc,Private jet carrying eight people crashes in M...,https://www.bbc.com/news/articles/cddgqqd0peno...
4,2026-01-26-16-46-51 +0000,startribune,The latest: Judge holds hearing on ICE surge a...,https://www.startribune.com/ice-raids-minnesot...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_3161/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
95,trump,33
35,shooting,23
184,minneapolis,21
32,minnesota,20
55,storm,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
201,2026-01-26-01-57-16 +0000,nypost,Trump ‘reviewing everything’ about fatal shoot...,https://nypost.com/2026/01/25/us-news/trump-re...,130
108,2026-01-26-12-22-24 +0000,nypost,Powerful House Republican suggests Trump consi...,https://nypost.com/2026/01/26/us-news/powerful...,126
51,2026-01-26-15-14-23 +0000,bbc,Trump says administration 'reviewing everythin...,https://www.bbc.com/news/articles/cr571qg4m61o...,93
112,2026-01-26-12-03-00 +0000,wsj,Republican unity over President Trump’s immigr...,https://www.wsj.com/politics/policy/miller-noe...,92
75,2026-01-26-14-24-51 +0000,nypost,Trump sending border czar Tom Homan to Minneso...,https://nypost.com/2026/01/26/us-news/trump-se...,89


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
201,130,2026-01-26-01-57-16 +0000,nypost,Trump ‘reviewing everything’ about fatal shoot...,https://nypost.com/2026/01/25/us-news/trump-re...
12,58,2026-01-26-16-34-00 +0000,wsj,"Deadly Winter Storm Disrupts Air Travel, Leave...",https://www.wsj.com/us-news/climate-environmen...
75,56,2026-01-26-14-24-51 +0000,nypost,Trump sending border czar Tom Homan to Minneso...,https://nypost.com/2026/01/26/us-news/trump-se...
74,42,2026-01-26-14-25-51 +0000,nypost,Final Israeli hostage in Gaza finally returned...,https://nypost.com/2026/01/26/world-news/body-...
19,39,2026-01-26-16-07-27 +0000,nypost,Stock market grew in 2025 — but less than any ...,https://nypost.com/2026/01/26/business/stock-m...
136,35,2026-01-26-11-00-00 +0000,startribune,Viewing violent ICE videos takes a toll. Here’...,https://www.startribune.com/videos-show-ice-ar...
225,32,2026-01-25-22-48-00 +0000,wsj,Senate Democrats angered by the deadly shootin...,https://www.wsj.com/politics/policy/senate-dem...
111,31,2026-01-26-12-10-00 +0000,wsj,Allied Gold Agreed to $4 Billion Buyout By Hon...,https://www.wsj.com/business/allied-gold-agree...
0,29,2026-01-26-17-01-08 +0000,nypost,NYC students struggle to sign in for virtual c...,https://nypost.com/2026/01/26/us-news/nyc-stud...
123,27,2026-01-26-11-23-20 +0000,bbc,Israel to reopen Gaza crossing after search fo...,https://www.bbc.com/news/articles/c801mj1n04lo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
